# Multi-Label Toxic Comment Classifier
Complete step-by-step pipeline using PyTorch + HuggingFace DistilBERT

In [16]:
# Step 1: Install required packages
import subprocess
import sys

packages = [
    "torch",
    "transformers",
    "pandas",
    "numpy",
    "scikit-learn",
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✓ All packages installed")

✓ All packages installed


In [17]:
# Step 2: Import libraries
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [18]:
# Step 3: Load the CSV file
df = pd.read_csv("train.csv")

print(f"✓ Dataset loaded")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst 3 rows:")
print(df.head(3))

✓ Dataset loaded
Shape: (159571, 8)
Columns: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

First 3 rows:
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  


In [19]:
# Step 4: Explore the data
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

print("Label statistics:")
print(df[label_cols].sum())
print(f"\nMissing values:")
print(df.isnull().sum())

Label statistics:
toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

Missing values:
id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [20]:
# Step 5: Preprocess text - handle NaN and strip whitespace
def preprocess_text(text):
    """Handle NaN values and strip whitespace"""
    if pd.isna(text):
        return ""
    return str(text).strip()

df["comment_text"] = df["comment_text"].apply(preprocess_text)

print("✓ Text preprocessing complete")
print(f"\nExample text:\n{df['comment_text'].iloc[0][:150]}")

✓ Text preprocessing complete

Example text:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted


In [21]:
# Step 6: Load tokenizer
MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 128

tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

print(f"✓ Tokenizer loaded: {MODEL_NAME}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Max length: {MAX_LENGTH}")

✓ Tokenizer loaded: distilbert-base-uncased
Vocab size: 30522
Max length: 128


In [22]:
# Step 7: Test tokenization on one example
sample_text = df["comment_text"].iloc[0]

sample_encoding = tokenizer(
    sample_text,
    padding="max_length",
    truncation=True,
    max_length=MAX_LENGTH,
    return_tensors="pt",
)

print("✓ Tokenization test on sample text")
print(f"Input text: {sample_text[:100]}...")
print(f"Input IDs shape: {sample_encoding['input_ids'].shape}")
print(f"Attention mask shape: {sample_encoding['attention_mask'].shape}")
print(f"Sample input IDs: {sample_encoding['input_ids'][0][:10].tolist()}")

✓ Tokenization test on sample text
Input text: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't ...
Input IDs shape: torch.Size([1, 128])
Attention mask shape: torch.Size([1, 128])
Sample input IDs: [101, 7526, 2339, 1996, 10086, 2015, 2081, 2104, 2026, 5310]


In [23]:
# Step 8: Create custom PyTorch Dataset class
class ToxicDataset(Dataset):
    """Custom Dataset for multi-label toxic comments"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.float32),
        }

print("✓ Custom Dataset class created")

✓ Custom Dataset class created


In [24]:
# Step 9: Prepare labels and create dataset
# Labels shape: (num_samples, 6) for 6 toxic label types
labels = df[label_cols].values.astype(np.float32)
texts = df["comment_text"].tolist()

print(f"Labels shape: {labels.shape}")
print(f"Texts count: {len(texts)}")
print(f"Sample labels: {labels[0]}")

# Create dataset
dataset = ToxicDataset(texts, labels, tokenizer, MAX_LENGTH)

print(f"\n✓ Dataset created with {len(dataset)} samples")

Labels shape: (159571, 6)
Texts count: 159571
Sample labels: [0. 0. 0. 0. 0. 0.]

✓ Dataset created with 159571 samples


In [25]:
# Step 10: Create DataLoader with default batch size
BATCH_SIZE = 16

train_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

print(f"✓ DataLoader created")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Number of batches: {len(train_loader)}")

# Test one batch
sample_batch = next(iter(train_loader))
print(f"\nSample batch shapes:")
print(f"  input_ids: {sample_batch['input_ids'].shape}")
print(f"  attention_mask: {sample_batch['attention_mask'].shape}")
print(f"  labels: {sample_batch['labels'].shape}")

✓ DataLoader created
  Batch size: 16
  Number of batches: 9974

Sample batch shapes:
  input_ids: torch.Size([16, 128])
  attention_mask: torch.Size([16, 128])
  labels: torch.Size([16, 6])


In [26]:
# Step 11: Setup device and load model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✓ Using device: {device}")

model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=6,
    problem_type="multi_label_classification",
)

model.to(device)

print(f"✓ Model loaded: {MODEL_NAME}")
print(f"Number of labels: 6")
print(f"Problem type: multi_label_classification")

✓ Using device: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Model loaded: distilbert-base-uncased
Number of labels: 6
Problem type: multi_label_classification


In [27]:
# Step 11b: Check GPU memory and find optimal batch size
print(f"\n✓ GPU Memory Check")
if torch.cuda.is_available():
    print(f"  Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"  Allocated memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"  Reserved memory: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    
    # Test different batch sizes to find maximum
    def test_batch_size(bs):
        """Test if batch_size fits in GPU memory"""
        try:
            test_loader = DataLoader(dataset, batch_size=bs, shuffle=False)
            sample_batch = next(iter(test_loader))
            
            input_ids = sample_batch["input_ids"].to(device)
            attention_mask = sample_batch["attention_mask"].to(device)
            
            with torch.no_grad():
                _ = model(input_ids=input_ids, attention_mask=attention_mask)
            
            torch.cuda.empty_cache()
            return True
        except RuntimeError:
            torch.cuda.empty_cache()
            return False
    
    print(f"\n✓ Testing batch sizes for RTX 3050...")
    max_batch_size = 16
    
    for bs in [16, 24, 32, 48, 64, 96, 128]:
        if test_batch_size(bs):
            max_batch_size = bs
            print(f"  Batch size {bs:3d}: ✓ OK")
        else:
            print(f"  Batch size {bs:3d}: ✗ OUT OF MEMORY")
            break
    
    # Recreate DataLoader with optimal batch size
    BATCH_SIZE = max_batch_size
    train_loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )
    
    print(f"\n✓ Updated DataLoader with optimal batch size: {BATCH_SIZE}")
    print(f"  Number of batches: {len(train_loader)}")
else:
    print("  CPU mode - no GPU optimization needed")


✓ GPU Memory Check
  Total GPU memory: 6.44 GB
  Allocated memory: 1.37 GB
  Reserved memory: 1.81 GB

✓ Testing batch sizes for RTX 3050...
  Batch size  16: ✓ OK
  Batch size  24: ✓ OK
  Batch size  32: ✓ OK
  Batch size  48: ✓ OK
  Batch size  64: ✓ OK
  Batch size  96: ✓ OK
  Batch size 128: ✓ OK

✓ Updated DataLoader with optimal batch size: 128
  Number of batches: 1247


In [28]:
# Step 12: Setup loss function
loss_fn = nn.BCEWithLogitsLoss()

print(f"✓ Loss function: BCEWithLogitsLoss")
print("  (suitable for multi-label classification)")

✓ Loss function: BCEWithLogitsLoss
  (suitable for multi-label classification)


In [29]:
# Step 13: Setup optimizer and scheduler
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps,
)

print(f"✓ Optimizer: AdamW")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"✓ Scheduler: Linear warmup")
print(f"  Total training steps: {total_steps}")

✓ Optimizer: AdamW
  Learning rate: 2e-05
✓ Scheduler: Linear warmup
  Total training steps: 3741


In [ ]:
# Step 14: Training loop with mixed precision (AMP) and timing
import time
from torch.cuda.amp import autocast, GradScaler

# Clear any leftover GPU memory
torch.cuda.empty_cache()

scaler = GradScaler() if torch.cuda.is_available() else None

print_once("training_start_sep", "="*70)
print_once("training_start_title", "STARTING TRAINING")
print_once("training_start_sep", "="*70)

model.train()
training_start = time.time()

for epoch in range(NUM_EPOCHS):
    print(f'\n--- Epoch {epoch+1}/{NUM_EPOCHS} ---')
    epoch_start = time.time()
    total_loss = 0.0

    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device, non_blocking=True)
        attention_mask = batch['attention_mask'].to(device, non_blocking=True)
        labels = batch['labels'].to(device, non_blocking=True)

        optimizer.zero_grad()

        if torch.cuda.is_available():
            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = loss_fn(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()

        total_loss += loss.item()

        # Print frequently to observe progress; reduce frequency if noisy
        if (batch_idx + 1) % 20 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            print(f'  Batch {batch_idx+1:4d} | Loss: {loss.item():.4f} | Avg Loss: {avg_loss:.4f}')

    epoch_time = time.time() - epoch_start
    epoch_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1} Complete | Avg Loss: {epoch_loss:.4f} | Time: {epoch_time:.1f}s')

total_time = time.time() - training_start
print_once("training_end_sep", '\n' + '='*70)
print_once("training_end_title", 'TRAINING COMPLETE')
print_once("training_end_sep", '='*70)
print(f'Total training time: {total_time:.1f} seconds ({total_time/60:.2f} minutes)')
print(f'Average time per epoch: {total_time/NUM_EPOCHS:.1f} seconds')


--- Epoch 1/3 ---
  Batch   20 | Loss: 0.0360 | Avg Loss: 0.0447
  Batch   20 | Loss: 0.0360 | Avg Loss: 0.0447
  Batch   40 | Loss: 0.0439 | Avg Loss: 0.0494
  Batch   40 | Loss: 0.0439 | Avg Loss: 0.0494
  Batch   60 | Loss: 0.0648 | Avg Loss: 0.0503
  Batch   60 | Loss: 0.0648 | Avg Loss: 0.0503
  Batch   80 | Loss: 0.0449 | Avg Loss: 0.0511
  Batch   80 | Loss: 0.0449 | Avg Loss: 0.0511
  Batch  100 | Loss: 0.0413 | Avg Loss: 0.0510
  Batch  100 | Loss: 0.0413 | Avg Loss: 0.0510
  Batch  120 | Loss: 0.0488 | Avg Loss: 0.0509
  Batch  120 | Loss: 0.0488 | Avg Loss: 0.0509
  Batch  140 | Loss: 0.0469 | Avg Loss: 0.0507
  Batch  140 | Loss: 0.0469 | Avg Loss: 0.0507
  Batch  160 | Loss: 0.0524 | Avg Loss: 0.0505
  Batch  160 | Loss: 0.0524 | Avg Loss: 0.0505
  Batch  180 | Loss: 0.0272 | Avg Loss: 0.0507
  Batch  180 | Loss: 0.0272 | Avg Loss: 0.0507
  Batch  200 | Loss: 0.0560 | Avg Loss: 0.0502
  Batch  200 | Loss: 0.0560 | Avg Loss: 0.0502
  Batch  220 | Loss: 0.0445 | Avg Loss: 0

In [40]:
# Step 15: Create output directories
model_dir = Path("toxic_model")
tokenizer_dir = Path("toxic_tokenizer")

model_dir.mkdir(exist_ok=True)
tokenizer_dir.mkdir(exist_ok=True)

print(f"✓ Output directories created")

✓ Output directories created


In [41]:
# Step 16: Save model
model.save_pretrained(model_dir)

print(f"✓ Model saved to: {model_dir}")
print(f"  Files: {list(model_dir.glob('*'))}")

✓ Model saved to: toxic_model
  Files: [WindowsPath('toxic_model/config.json'), WindowsPath('toxic_model/model.safetensors')]


In [42]:
# Step 17: Save tokenizer
tokenizer.save_pretrained(tokenizer_dir)

print(f"✓ Tokenizer saved to: {tokenizer_dir}")
print(f"  Files: {list(tokenizer_dir.glob('*'))}")

✓ Tokenizer saved to: toxic_tokenizer
  Files: [WindowsPath('toxic_tokenizer/special_tokens_map.json'), WindowsPath('toxic_tokenizer/tokenizer.json'), WindowsPath('toxic_tokenizer/tokenizer_config.json'), WindowsPath('toxic_tokenizer/vocab.txt')]


In [43]:
# Step 18: Create inference function
def predict(text, model_path="toxic_model", tokenizer_path="toxic_tokenizer"):
    """
    Predict toxicity probabilities for input text.
    
    Args:
        text (str): Input comment text
        model_path (str): Path to saved model
        tokenizer_path (str): Path to saved tokenizer
    
    Returns:
        dict: Text and probabilities for each label
    """
    # Load model and tokenizer
    loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
    loaded_tokenizer = DistilBertTokenizerFast.from_pretrained(tokenizer_path)
    
    loaded_model.eval()
    loaded_model.to(device)
    
    # Preprocess and tokenize
    text = preprocess_text(text)
    inputs = loaded_tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt",
    )
    
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    # Predict
    with torch.no_grad():
        outputs = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    
    # Return probabilities for each label
    label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    return {
        "text": text[:100] + "..." if len(text) > 100 else text,
        "probabilities": {label: float(prob) for label, prob in zip(label_cols, probs)},
    }

print("✓ Inference function created")

✓ Inference function created


In [44]:
# Step 19: Test inference on sample texts
print("="*70)
print("TESTING INFERENCE")
print("="*70)

test_texts = [
    "This is a great movie! I loved it.",
    "You are an idiot and should die.",
    "I hate people like you, they are disgusting.",
    "The weather is nice today.",
    "I hope you get hit by a car.",
]

for i, test_text in enumerate(test_texts, 1):
    print(f"\n--- Test {i} ---")
    result = predict(test_text)
    print(f"Input: {result['text']}")
    print("Probabilities:")
    for label, prob in result['probabilities'].items():
        bar = "█" * int(prob * 20)
        print(f"  {label:15s}: {prob:.4f} {bar}")

TESTING INFERENCE

--- Test 1 ---
Input: This is a great movie! I loved it.
Probabilities:
  toxic          : 0.0016 
  severe_toxic   : 0.0001 
  obscene        : 0.0004 
  threat         : 0.0001 
  insult         : 0.0002 
  identity_hate  : 0.0001 

--- Test 2 ---
Input: You are an idiot and should die.
Probabilities:
  toxic          : 0.9911 ███████████████████
  severe_toxic   : 0.4456 ████████
  obscene        : 0.7602 ███████████████
  threat         : 0.8043 ████████████████
  insult         : 0.9155 ██████████████████
  identity_hate  : 0.0931 █

--- Test 3 ---
Input: I hate people like you, they are disgusting.
Probabilities:
  toxic          : 0.9819 ███████████████████
  severe_toxic   : 0.0148 
  obscene        : 0.0285 
  threat         : 0.0383 
  insult         : 0.3978 ███████
  identity_hate  : 0.1695 ███

--- Test 4 ---
Input: The weather is nice today.
Probabilities:
  toxic          : 0.0004 
  severe_toxic   : 0.0000 
  obscene        : 0.0002 
  threat         